## Exercises week 2

# Working with textual data

### 0. Get the data.

- Download  `articles.tar.gz` or `articles.zip` from Canvas (under `Week 2`). Please note that this is not the full dataset, but random sample of the data described [here](https://dx.doi.org/10.7910/DVN/ULHLCB).


<div class="alert-danger">
<p>Alternatively, you can also download <code>articles.tar.gz</code> from
<a href="https://dx.doi.org/10.7910/DVN/ULHLCB">https://dx.doi.org/10.7910/DVN/ULHLCB</a> to get <strong>all</strong> the data. Please note that this is a very large dataset, and for practice purposes, you do not need everything. </p>
</div>



- Unpack it. On Linux and MacOS, you can do this with `tar -xzf mydata.tar.gz` on the command line. On Windows, you may need an additional tool such as `7zip` for that (note that technically speaking, there is a `tar` archive within a `gz` archive, so unpacking may take *two* steps depending on your tool).


### 1. Inspect the structure of the dataset.
What information do the following elements give you?

- folder (directory) names
- folder structure/hierarchy
- file names
- file contents

### 2. Discuss strategies for working with this dataset!

- Which questions could you answer?
- How could you deal with it, given the size and the structure?

### 3. Read some (or all?) data

Here is some example code that you can modify. you could, for instance, do the following to read a *part* of your dataset.

In [ ]:
from glob import glob
import random
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

#specify the path to your unpacked articles.
PATH = 'path/to/articles'

In [ ]:
newspaperfiles = glob(PATH+'/*/Vox/*')
documents = []
for filename in newspaperfiles:
    with open(filename) as f:
        documents.append(f.read())

In [ ]:
len(documents)

<div class="alert-info">
<ul>
<li>Can you explain what the <code>glob</code> function does?</li>
<li>What does <code>documents</code> contain? First make an educated guess based on the code snippet, then check it! Do <em>not</em> print the whole thing, but use <code>len()</code>, <code>type()</code> en slicing (e.g.,<code>[:10]</code>) to get the info you need.</li>
</ul>
</div>


<br>
<div class="alert-block alert-warning">
<p>Tip: take a random sample of the articles for practice purposes (if your code works, you can scale up!)</p><code>articles =random.sample(documents, 10)</code></p>
</div>

### 4. Perform some analyses!

- Perform some first analyses on the data using string methods and regular expressions!

Techniques you can try out include:

a.  lowercasing

b.  tokenization

c.  stopword removal

d.  stemming and/or lemmatizing

In [ ]:
articles =random.sample(documents, 10)

    #a. lowercasing articles

In [ ]:
articles_lower_cased = [art.lower() for art in articles]

    #b. tokenization

In [ ]:
#b. tokenization: basic
articles_split = [art.split() for art in articles]

#. tokenization: more advanced
articles_tokenized = [TreebankWordTokenizer().tokenize(art) for art in articles ]

    #c. Stopword removal

In [ ]:
# mystopwords = stopwords.words("english")
mystopwords = stopwords.words("english")

In [ ]:
mystopwords.extend(["add", "more", "words"]) # manually add more stopwords to your list if needed
print(mystopwords)

In [ ]:
#Now, remove stopwords from the corpus:
    
articles_without_stopwords = []
for article in articles:
    articles_no_stop = ""
    for word in article.lower().split():
        if word not in mystopwords:
            articles_no_stop = articles_no_stop + " " + word
    articles_without_stopwords.append(articles_no_stop)

In [ ]:
# same as the cell above, this time with list comprehension
articles_without_stopwords = [" ".join([w for w in article.lower().split() if w not in mystopwords]) for article in articles]


<br>
<div class="alert-block alert-warning">
It's good practice to frequently inspect the results of your code, to make sure you are not making mistakes, and the results make sense. For example, compare your results to some random articles from the original sample:

In [ ]:
print(articles[8][:100])
print("-----------------")
print("".join(articles_without_stopwords[8])[:100])

    #4. Stemming

In [ ]:
stemmer = SnowballStemmer("english")

stemmed_text = []
for article in articles:
    stemmed_words = ""
    for word in article.lower().split():
        stemmed_words = stemmed_words + " " + stemmer.stem(word)
    stemmed_text.append(stemmed_words.strip())

In [ ]:
# same as the cell above, this time with list comprehension

In [ ]:
stemmed_text  = [" ".join([stemmer.stem(w) for w in article.lower().split()]) for article in articles]

## 5. Vectorize the data and create a word cloud

<div class="alert-info">
<ul>
<p>a. Think about different ways to vectorize the data. You can use <code>count</code> and / or <code>tfidf</code><br>b. Also consider different parameter settings of the vectorizers. For example, apply pruning or not.</p>

In [ ]:
# initialize the vectorizer
vec = CountVectorizer()
# fit and transform the documents
vec_fit = vec.fit_transform(articles_without_stopwords)
vec_fit.shape

In [ ]:
vec.get_feature_names()

In [ ]:
# transform the data from sparse to dense: do not do this with a very large dataset. 
feature_names = vec.get_feature_names()
dense = vec_fit.todense()
lst1 = dense.tolist()
df = pd.DataFrame(lst1, columns=feature_names)
df.T.sum(axis=1)

In [ ]:
Cloud = WordCloud(background_color="black", max_words=50).generate_from_frequencies(df.T.sum(axis=1))

plt.imshow(Cloud)
plt.axis("off")
plt.show()

<div class="alert-info">
<ul>
<ul>
<p>a. Do the results change if you use <code>tfidf</code> transformation?<br>b. What happens if you use different pre-processing decisions? How does that affect the final word cloud?<br>c. What happens if include all the data?</p>
</ul>
<ul>